In [2]:
import os
import random
import torch
import mne

import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.utils import shuffle
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
from torch.utils.data import random_split
from torch_geometric.utils import to_undirected
from scipy.spatial.distance import cdist
from scipy.stats import skew, kurtosis
from tqdm import tqdm


### Brainstrom, what we can do ?
* first, classification
* then, try transfer learning ?
* more deeply, apply time-frequency analysis
    * classification
    * transfer learning


### some basic funcs

In [3]:
def train(model, train_loader, optimizer, criterion):
    model.train()
    tol_loss = 0
    
    for batch in tqdm(train_loader, desc="Training"):
        ## zero gradient
        optimizer.zero_grad()
        ## forward pass
        out = model.forward(batch.x, batch.edge_index, batch.batch)
        ## calculate loss according to output and y
        loss = criterion(out, batch.y)
        ## backward propagation
        loss.backward()
        ## do optimization
        optimizer.step()
        ## accumulate loss
        tol_loss += loss.item()
    ## return the mean loss    
    return tol_loss / len(train_loader)

def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    
    ## no need to do gradient descent in test phase  
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Testing"):
            ## forward pass
            out = model.forward(batch.x, batch.edge_index, batch.batch)
            prediction = out.argmax(dim = 1)
            correct += (prediction == batch.y).sum().item()
            total += batch.y.size(0)
            
        accuracy = correct / total
    return  accuracy   

def fit(model, train_loader, test_loader, epochs = 200, lr = 0.01):
        ## define optimizer and criterion
        optimizer = torch.optim.Adam(model.parameters(), lr = lr)
        ## here i choose cross entropy
        criterion = nn.CrossEntropyLoss()
        
        for epoch in range(epochs):
                train_loss = train(model, train_loader, optimizer, criterion)  
                test_acc = test(model, test_loader)
                print(f'Epoch {epoch:03d}, Train Loss: {train_loss:.4f}, Test Acc: {test_acc:.4f}')
        
        return test_acc
    
def use_method(dist_mat, method, thres = 0.05, k = 4):
    '''
    Choose the method to build adjacency mat
    
    Args:
        dist_mat: matrix; a matrix has been calculated the Euclidean distance
        method: str; kkn or thres
        
    Returns:
        adjacency matrix
    '''
    adj_mat = np.zeros_like(dist_mat)
    
    if method == 'thres':
        # thres will set a threshold for the whole mat, higher →1，lower→0 
        adj_mat = np.where(dist_mat <= thres, 1, 0) 
        
    elif method == 'knn':
        # knn will find the top nearest 5 channel for each 
        for i in range(dist_mat.shape[0]):
            idx = np.argsort(dist_mat[i])[0: k+1]  # sort in row i
            adj_mat[i, idx] = 1  
        adj_mat = np.maximum(adj_mat, adj_mat.T)  
    return adj_mat  


### build a simple CNN

#### define CNN first
CNN1 → ReLU → CCN2 → ReLU → pool → Linear1 → ReLU → Dropout(0.5) → Linear2 → ReLU → Output Layer（classification）

In [4]:
class SimpleCCN(nn.Module):
    def __init__(self):
        '''
        Initialization params
        
        Args:
            None
        Returns:
            Nothing
        '''
        ## allocate the nn.module
        
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 4, out_channels = 32, kernel_size = 3)
        self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 3, kernel_size = 3)
        self.pool = nn.MaxPool2d(kernel_size = 2)
        self.fc1 = nn.Linear(in_features = 9216, out_features = 9216)
        self.fc2 = nn.Linear(in_features = 9216, out_features = 128)
        self.fc3 = nn.Linear(in_features = 128, out_features = 3)
    
    def forawrd(self):
        '''
        Here is a forward pass
        
        Args:
            x: torch.tensor
               Input feature   
                   
        Returns:
            x: torch.tensor
               Output of final fully connected layer
        '''
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.dropout(x)
        x = F.relu(x)
        x = self.fc2(x) 
        x = F.relu(x)
        
        return x

#### what should the input vector look like??

### build a simple GCN

####  define GCN first
* GCN1 → ReLU → GCN2 → ReLU → meanpool → Linear1 → ReLU → Linear2 → ReLU → Output Layer（classification）

In [5]:
class SimpleGCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, fc_hidden, num_classes):
        '''
        Args:
            in_channels: dimension of input
            hidden_channels: numbers of hidden_layer
            fc_hidden: dimension of input
            num_classes: numbers of classification
        '''
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)  
        self.fc1 = nn.Linear(hidden_channels, fc_hidden)
        self.fc2 = nn.Linear(fc_hidden, fc_hidden)
        self.classifier = nn.Linear(fc_hidden, num_classes)
        self.dropout = nn.Dropout(0.5) 
        
    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = global_mean_pool(x, batch=batch)
        
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)

        x = self.fc2(x)
        x = F.relu(x)

        x = self.classifier(x)
        
        return x
                

#### ready for the dataset
* channels and sequences

    'Fp1', 'Fpz', 'Fp2', 'AF7', 'AF3','AF4','AF8', 'F7', 'F5','F3','F1','Fz', 'F2', 'F4', 'F6', 'F8', 'FT7', 'FC5', 'FC3', 'FC1','FCz','FC2','FC4', 'FC6', 'FT8', 'T7','C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1','CPz','CP2', 'CP4','CP6', 'TP8', 'P7','P5', 'P3', 'P1', 'Pz','P2', 'P4', 'P6', 'P8', 'PO7', 'PO3','POz', 'PO4','PO8', 'O1','Oz','O2', 'F9', 'F10', 'TP9', 'TP10'
* After reordering, the sequence for both imagery and video trials is as follows:

    reorder = ['sad4', 'sad5', 'sad8', 'dis4', 'dis5', 'dis8', 'fear4', 'fear5', 'fear8', 'neu4', 'neu5', 'neu8', 'joy4', 'joy5', 'joy8', 'ten4', 'ten5', 'ten8', 'ins4', 'ins5', 'ins8']
* emotion labels: 
    * negative(sadness, disgust, fear):             0
    * positive(happiness, inspiration, tenderness)：1
    * neutral:                                      2

* so the reoder labels sequence should be: [0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [6]:
##--------------------------load mne------------------------------##
# load biosemi64 electrode layout
montage = mne.channels.make_standard_montage('standard_1005')
positions = montage.get_positions()

## electrode name and 3-dimensional coordinates
ch_pos = positions['ch_pos']

df = pd.DataFrame.from_dict(ch_pos, orient = 'index', columns = ['x', 'y','z'])
df = df.reset_index().rename(columns = {'index': 'electrode'})
# print (df)

##------------------------load channels --------------------------##
channels = ['Fp1', 'Fpz', 'Fp2', 'AF7', 'AF3', 'AF4', 'AF8', 'F7',
            'F5',  'F3',  'F1',  'Fz',  'F2',  'F4',  'F6',  'F8', 
            'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 
            'FT8', 'T7',  'C5',  'C3',  'C1',  'Cz',  'C2',  'C4', 
            'C6',  'T8',  'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2',
            'CP4', 'CP6', 'TP8', 'P7',  'P5',  'P3',  'P1',  'Pz',
            'P2',  'P4',  'P6',  'P8',  'PO7', 'PO3', 'POz', 'PO4',
            'PO8', 'O1',  'Oz',  'O2',  'F9',  'F10', 'TP9', 'TP10']

##-----------------calculate Euclidean distance----------------------##
# p1 = np.array([x1, y1, z1])
# distance = cdist(mat1, mat2)
used_channel = channels[: 64]

# build a map 
coord_map = {ch: ch_pos[ch] for ch in used_channel}
coords = np.array([coord_map[ch] for ch in used_channel])

# now calculate the distance
dist_mat = cdist(coords, coords)
# dist_df = pd.DataFrame(dist_mat, index=used_channel, columns=used_channel)
dist_tensor = torch.tensor(dist_mat, dtype=torch.float)

##-------------------build feature matrix-----------------------------##
data = np.load('EEG_data\sub-01_ses-ima_task-emotion_reorder.npy')
data = np.transpose(data, (1, 0, 2))
num_trials, num_channels, num_samples = data.shape
features = np.zeros((num_trials, num_channels, 4))
features[:, :, 0] = np.mean(data, axis=2)
features[:, :, 1] = np.std(data, axis=2)
features[:, :, 2] = skew(data, axis=2)
features[:, :, 3] = kurtosis(data, axis=2)
features_tensor = torch.tensor(features, dtype=torch.float) 



In [7]:
## adjacency mat
adj_mat = torch.tensor(use_method(dist_mat, 'knn', k=5), dtype=torch.float )  

## feature mat 
feature_mat = features_tensor   # feature， Should be the exact value([mean, std, skewness, kurtosis])

y = torch.tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1])  # real emotion 

# connection
edge_index_np = np.array(np.nonzero(adj_mat))
edge_index = torch.tensor(edge_index_np, dtype=torch.long)
edge_index = to_undirected(edge_index)
# split Data into train_set and test_set
input_lst = []
for i in range(len(y)):
    x = feature_mat[i]
    y_i = y[i]
    data_i = Data(x=x, edge_index=edge_index.clone(), y=y_i)  ## remember to clone!
    input_lst.append(data_i)

train_len = int(0.8 * len(input_lst))
test_len = len(input_lst) - train_len
train_dataset, test_dataset = random_split(input_lst, [train_len, test_len])

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4)

####  now run this model

In [8]:
model = SimpleGCN(in_channels=4, hidden_channels=64, fc_hidden=128, num_classes=3)
res_acc = fit(model, train_loader=train_loader, test_loader=test_loader, epochs=10, lr=0.01)


Testing: 100%|██████████| 2/2 [00:00<00:00, 388.38it/s]


Epoch 000, Train Loss: 1.1710, Test Acc: 0.4000


Testing: 100%|██████████| 2/2 [00:00<00:00, 259.13it/s]


Epoch 001, Train Loss: 1.1536, Test Acc: 0.4000


Testing: 100%|██████████| 2/2 [00:00<00:00, 349.60it/s]


...

Epoch 007, Train Loss: 1.0768, Test Acc: 0.4000


Testing: 100%|██████████| 2/2 [00:00<00:00, 370.88it/s]


Epoch 008, Train Loss: 1.0394, Test Acc: 0.6000


Testing: 100%|██████████| 2/2 [00:00<00:00, 328.00it/s]

Epoch 009, Train Loss: 1.0516, Test Acc: 0.6000
